In [2]:
from soilgrids import SoilGrids

“get_coverage_data()” method includes multiple parameters for data download. Details for each parameter are listed below.

- **service_id**: The identifier of each map service provided by the SoilGrids system. The supported service id and the corresponding variable names are shown below. The “map_services” attribute of an instance will show more detailed information.
    - **bdod**: Bulk density
    - **cec**: Citation exchange capacity at ph7
    - **cfvo**: Coarse fragments volumetric
    - **clay**: Clay content
    - **nitrogen**: Nitrogen
    - **phh2o**: Soil pH in H2O
    - **sand**: Sand content
    - **silt**: Silt content
    - **soc**: Soil organic carbon content
    - **ocs**: Soil organic carbon stock
    - **ocd**: Organic carbon densities
    
- **coverage_id**: The identifier of a coverage(map) from a map service. Each map service supports multiple coverages. To get a list of the coverage id from a map service, use “get_coverage_list()” method. To learn the meaning of the coverage id, please visit the SoilGrids FAQ page

- **crs**: the coordinate system code of a coverage. To get the supported crs code list of a coverage, use “get_coverage_info()” method.

- **west, south, east, north**: The bounding box values for the downloaded data. These values should be based on the coordinate system specified by the “crs” parameter. The west and south values are for the point on the lower left corner of the bounding box. The east and north values are for the point on the upper right corner of the bounding box.

- **output**: The file path of the GeoTiff file to store the downloaded data with “.tif” file extension.

- **resx, resy**: The grid resolution for the downloaded data when “crs” parameter is set as a projection coordinate system(e.g., epsg 152160). The default value for resx and resy is set as 250 (m) if not specified by the user. This is the same grid resolution as the soil datasets in the SoilGrids system. The resx and resy parameters are required when the “crs” parameter is set as a projection coordinate system and the width and height values are not needed.

- **width, height**: The width and height of the raster for the downloaded data when “crs” parameter is set as a geographic coordinate system(e.g., epsg 4326 for WGS84). The height represents the number of rows and the width represents the number of columns for the raster grid of the downloaded data. The width and height parameters are required when the “crs” is set as a geographic coordinate system and resx and resy values are not needed.

- **response_crs**: the coordinate system code for the GeoTiff file of the downloaded data. If response_crs is not specified by the user, its value will be the same as the crs value.

- **local_file**: indicate whether to make it priority to get the data by loading a local file that matches with the output file path. Default value is set as False, which means the function will directly download the data from SoilGrids system. If value is set as True, the function will first try to open a local file that matches with the output file path. And if the local file doesn’t exist, it will then download data from SoilGrids.

In [3]:


# get data from SoilGrids
soil_grids = SoilGrids()
data = soil_grids.get_coverage_data(service_id='bdod', coverage_id='phh2o_0-5cm_mean',
                                       west=-1784000, south=1356000, east=-1140000, north=1863000,
                                       crs='urn:ogc:def:crs:EPSG::152160',output='test.tif')

# show metadata
for key, value in soil_grids.metadata.items():
    print('{}: {}'.format(key,value))


variable_name: Soil pH in H2O
variable_units: pH*10
service_url: https://maps.isric.org/mapserv?map=/map/phh2o.map
service_id: phh2o
coverage_id: phh2o_0-5cm_mean
crs: urn:ogc:def:crs:EPSG::152160
bounding_box: (-1784000, 1356000, -1140000, 1863000)
grid_res: [250, 250]


In [4]:
data

<xarray.DataArray (band: 1, y: 2028, x: 2576)>
[5224128 values with dtype=int16]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 -1.784e+06 -1.784e+06 ... -1.14e+06 -1.14e+06
  * y            (y) float64 1.863e+06 1.863e+06 ... 1.356e+06 1.356e+06
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  2 (pixels/inch)
    TIFFTAG_XRESOLUTION:     72
    TIFFTAG_YRESOLUTION:     72
    scale_factor:            1.0
    add_offset:              0.0

In [7]:
import rasterio
import geopandas as gpd

# Read points from shapefile
pts = gpd.read_file("./wosis_latest/wosis_latest_clay_Argentina.gpkg")


DriverError: database is locked

In [ ]:
pts = pts[['UTM_E', 'UTM_N', 'Value', 'geometry']]
pts.index = range(len(pts))
coords = [(x,y) for x, y in zip(pts.UTM_E, pts.UTM_N)]

# Open the raster and store metadata
src = rasterio.open('your_raster.tif')

# Sample the raster at every point location and store values in DataFrame
pts['Raster Value'] = [x for x in src.sample(coords)]
pts['Raster Value'] = probes.apply(lambda x: x['Raster Value'][0], axis=1)